# Оценка возможности прогнозирования ДТП

Цель работы заключается в создании системы для каршеринговой компании, которая могла бы оценить риск ДТП по совокупности факторов. Под риском понимается вероятность ДТП с любым повреждением транспортного средства. Как только водитель забронировал автомобиль, сел за руль и выбрал маршрут, система должна оценить уровень риска. Если уровень риска высок, водитель увидит предупреждение и рекомендации по маршруту.

Идея создания такой системы находится в стадии предварительного обсуждения и проработки. Чёткого алгоритма работы и подобных решений на рынке ещё не существует. Текущая задача — понять, возможно ли предсказывать виновность в ДТП, опираясь на исторические данные одного из регионов.

Идея решения задачи от заказчика:

Создать модель предсказания ДТП (целевое значение — at_fault (виновник) в таблице parties)
    Для модели выбрать тип виновника — только машина (car).
    Выбрать случаи, когда ДТП привело к любым повреждениям транспортного средства, кроме типа SCRATCH (царапина).
    Для моделирования ограничиться данными за 2012 год — они самые свежие.
    Обязательное условие — учесть фактор возраста автомобиля.
На основе модели исследовать основные факторы ДТП.
Понять, помогут ли результаты моделирования и анализ важности факторов ответить на вопросы:
    Возможно ли создать адекватную системы оценки водительского риска при выдаче авто?
    Какие ещё факторы нужно учесть?
    Нужно ли оборудовать автомобиль какими-либо датчиками или камерой?
    
Таким образом, исследование будет состоять из следующих этапов:

обзор данных;

предобработка данных;

исследовательский анализ данных;

проведение статистического анализа факторов ДТП;

разработка ML - модели оценки водительского риска;

анализ важности факторов ДТП;

основные выводы.

Краткое описание таблиц

collisions — общая информация о ДТП. Имеет уникальный case_id. Эта таблица описывает общую информацию о ДТП. Например, где оно произошло и когда.

parties — информация об участниках ДТП. Имеет неуникальный case_id, который сопоставляется с соответствующим ДТП в таблице collisions. Каждая строка здесь описывает одну из сторон, участвующих в ДТП. Если столкнулись две машины, в этой таблице должно быть две строки с совпадением case_id. Если нужен уникальный идентификатор, это case_id and party_number.

vehicles — информация о пострадавших машинах. Имеет неуникальные case_id и неуникальные party_number, которые сопоставляются с таблицей collisions и таблицей parties. Если нужен уникальный идентификатор, это case_id and party_number.

## Подключитесь к базе. Загрузите таблицы sql

## Загрузка библиотек

In [ ]:
!pip install phik -q
!pip install sweetviz -q

In [ ]:
!pip install pandas_profiling -q
!pip install shap -q

In [ ]:
!pip install -U scikit-learn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import statistics as st
import sweetviz as sv
import phik
from phik.report import plot_correlation_matrix
from phik import report


import math
import random
import shap
import time
import xgboost


from sqlalchemy import create_engine
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder,  LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV



from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

SEED = 42

pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Подключение к серверу

Объявим конфигурацию для подключения к серверу.

In [ ]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-science-vehicle-db' # название базы данных
}

Формируем строку для подключения.

In [ ]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

Создадим соединение.

In [ ]:
engine = create_engine(connection_string)

Создадим пробный запрос.

In [ ]:
query = '''
SELECT COUNT(DISTINCT id) AS count_parties
  FROM parties
 WHERE at_fault = 1;
'''
df = pd.read_sql_query(query, con=engine)
df.head()

Вывод:

Проведено успешное подключение к серверу и выполнен подсчет всех уникальных виновников ДТП. Перейдём к исследованию таблиц.

## Проведите первичное исследование таблиц

Посмотрим на размер БД.

In [ ]:
query = '''
SELECT pg_size_pretty(pg_database_size(current_database()));
'''
df = pd.read_sql_query(query, con=engine)
df

Посмотрим названия таблиц в БД.

In [ ]:
query = '''
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema NOT IN ('information_schema','pg_catalog');
'''
df = pd.read_sql_query(query, con=engine)
df

Посмотрим какие колонки имеются в наших таблицах и какой тип данных они имеют.

In [ ]:
query = '''
SELECT table_name,
       column_name,
       data_type
  FROM information_schema.columns
 WHERE table_name IN ('case_ids', 'collisions', 'parties', 'vehicles');
'''
df = pd.read_sql_query(query, con=engine)
df

Посмотрим информацию по таблицам.

In [ ]:
collisions = '''

SELECT *
FROM collisions

'''

parties = '''

SELECT *
FROM parties

'''

vehicles = '''

SELECT *
FROM vehicles

'''

case_ids = '''

SELECT *
FROM case_ids

'''

Выгрузим таблицу collisions в датафрейм и запросим информацию о ней.

In [ ]:
collisions = pd.read_sql_query(collisions, con = engine)

In [ ]:
collisions.info()

In [ ]:
collisions.head(5)

In [ ]:
collisions.describe()

Выгрузим таблицу parties в датафрейм и запросим информацию о ней.

In [ ]:
parties = pd.read_sql_query(parties, con = engine)

In [ ]:
parties.info()

In [ ]:
parties.head(5)

In [ ]:
parties.describe()

Выгрузим таблицу vehicles в датафрейм и запросим информацию о ней.

In [ ]:
vehicles = pd.read_sql_query(vehicles, con = engine)

In [ ]:
vehicles.info()

In [ ]:
vehicles.head(5)

In [ ]:
vehicles.describe()

Выгрузим таблицу case_ids в датафрейм и запросим информацию о ней.

In [ ]:
case_ids = pd.read_sql_query(case_ids, con = engine)

In [ ]:
case_ids.info()

In [ ]:
case_ids.head(5)

Похоже, что в таблице collisions отсутствует первичный ключ. Проверим уникальность значений столбца case_id.

In [ ]:
query = '''
SELECT COUNT(case_id) AS total_ids,
       COUNT(DISTINCT case_id) AS unique_ids
  FROM collisions;
'''
df = pd.read_sql_query(query, con=engine)
df

Значения уникальны.

Также стоит отметить, что первичный ключ в таблицах parties и vehicles не совпадает с внешним ключом к таблице case_ids, т.к. одному case_id может быть сопоставлено несколько id из этих таблиц.

Выводы

Нам представлена БД с описанием дорожно транспортных проишествий на 4 таблицы, одна из которых является связывающей (case_ids), остальные - информационные объекты, которые несут в себе характиристики машины (Vehicles), участников (Parties) и самого проишествия (collisions). Все таблицы имеют данные, необходимые для обучения моделей и решения поставленной задачи. Наблюдений в каждой сущности свыше 1.000.000, что будет лишь положительно влиять на нашу тренировку. Мы осмотрели данные и убедились что с ними всё в порядке. Содержимое некоторых столбцов не использует буквенные сокращения, как представлено в описании. Перейдём к статистическому анализу.

##  Проведите статистический анализ факторов ДТП

## Выявление наиболее аварийных месяцев

Выясним, в какие месяцы происходит наибольшее количество аварий. Для этого выгрузим нужные столбцы из таблицы collisions, посчитаем количество case_id и сгруппируем по месяцам.

In [ ]:
query = '''

SELECT DISTINCT extract(MONTH FROM cast(collision_date AS date))::int AS collision_month,
                count(case_id) AS case_count
FROM collisions
GROUP BY 1
'''
coll_count_df = pd.read_sql_query(query, con=engine)
coll_count_df

Построим график по полученной таблице:

In [ ]:
plt.figure(figsize=[15,6])
sns.lineplot(data=coll_count_df, x='collision_month', y='case_count', label='Число ДТП', color='red')
plt.legend()
plt.title('Число ДТП по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Количество ДТП')
plt.grid()
plt.show()

Наибольшее количество ДТП наблюдается в период с января по май, дальше идет резкое снижение, пик приходится на март. Такой всплеск может быть связан с неблагоприятными погодными условиями и состоянием дорог в зимне-весенний период или с разным количеством данных по месяцам, что необходимо проверить.

In [ ]:
query = '''
SELECT COUNT(case_id) AS total_ids,
       DATE_TRUNC('month', collision_date)::date AS month
FROM collisions
GROUP BY DATE_TRUNC('month', collision_date)
ORDER BY DATE_TRUNC('month', collision_date);
'''
df = pd.read_sql_query(query, con=engine)

In [ ]:
fig = px.bar(
    df, x='month', y='total_ids',
    barmode='group',
    title=('Статистика происшествий по месяцам')
)
fig.show()

Из графика видно, что статистика собрана в полной мере только до мая 2012, затем в данных существенный провис.

Если рассматривать зону с большим покрытием (до мая 2012), то видно несколько трендов:

меньшее количество аварий происходит в январе-феврале;

пик аварий все три года приходится на октябрь, что может быть связано с первыми заморозками и снегом, а также не успевшими адаптироваться водителями.

## Постановка задач для рабочей группы

Список задач:

1. Определить серьёзность повреждений ТС в зависимости от года выпуска автомобиля (таблицы vehicles и collisions).

2. Оценить серьёзность повреждений ТС, исходя из состояния водителя (таблицы parties и collisions).

3. Автомобили какого типа чаще всего попадают в дорожно-транспортные происшествия. Какие типы нарушений для них являются причинами аварий (таблицы vehicles и collisions).

4. Серьёзность повреждений ТС в зависимости от года выпуска автомобиля (таблицы vehicles и collisions).

5. Найти ДТП с числом участников больше 2, в которых водитель разговаривал по телефону (таблицы parties и collisions).

6. Как связаны качество дорожного покрытия и тип коробки передач автомобиля, попавшего в аварию (таблицы vehicles и collisions).

Решение задачи "Определить серьёзность повреждений ТС в зависимости от года выпуска автомобиля".

In [ ]:
query = '''
SELECT c.collision_damage,
       v.vehicle_age
  FROM collisions AS c
       JOIN vehicles AS v ON c.case_id = v.case_id;
'''
df = pd.read_sql_query(query, con=engine)

Уберем возраст автомобилей старше 20 лет.

In [ ]:
df.drop(index=df[df['vehicle_age'] > 20].index, inplace=True)

In [ ]:
fig = px.histogram(
    df, x='vehicle_age', color='collision_damage',
    barmode='group',
    title='Зависимость тяжести ДТП от возраста ТС (количество)',
    histfunc='count',
    category_orders={
        'collision_damage': ['fatal', 'severe damage', 'middle damage', 'small damage', 'scratch']
    }
)
fig2 = px.histogram(
    df, x='vehicle_age', color='collision_damage',
    barmode='group',
    title='Зависимость тяжести ДТП от возраста ТС (проценты)',
    histfunc='count',
    histnorm='percent',
    category_orders={
        'collision_damage': ['fatal', 'severe damage', 'middle damage', 'small damage', 'scratch']
    }
)
fig.show()
fig2.show()

Из гистограмм мы можем сделать следующие выводы:

Начиная с третьего года владения ТС доля аварий с незначительными повреждениями scratch сильно вырастает, что может быть связано с тем что автомобилисты бережней относятся к явно более новым машинам.

Начиная с четвертого года владения увеличивается доля ДТП с фатальными повреждениями.

С возрастом постепенно меняется соотношение типов аварий, становится меньше лёгких и больше тяжёлых аварий, возможно, сказывается техническое состояние автомобилей.

Решение задачи "Оценить серьёзность повреждений ТС, исходя из состояния водителя".

In [ ]:
query = '''
SELECT c.collision_damage,
       p.party_drug_physical
FROM collisions AS c
JOIN parties AS p ON c.case_id = p.case_id
WHERE p.party_drug_physical IN ('not applicable', 'G');
'''
df = pd.read_sql_query(query, con=engine)

In [ ]:
fig = px.histogram(
    df, x='party_drug_physical', color='collision_damage',
    barmode='group',
    title='Зависимость тяжести ДТП от физического состояния водителя (количество)',
    histfunc='count',
    category_orders={
        'collision_damage': ['fatal', 'severe damage', 'middle damage', 'small damage', 'scratch']
    }
)
fig.show()

В случаях, когда состояние о физическом состоянии неизвестно или не оценивалось - статистика видов повреждения очень похожа. Большая часть аварий заканчивается малыми повреждениями или царапинами.

In [ ]:
query = '''
SELECT c.collision_damage,
       p.party_drug_physical
FROM collisions AS c
JOIN parties AS p ON c.case_id = p.case_id
WHERE p.party_drug_physical IN ('under drug influence', 'sleepy/fatigued', 'impairment - physical');
'''
df = pd.read_sql_query(query, con=engine)

In [ ]:
fig = px.histogram(
    df, x='party_drug_physical', color='collision_damage',
    barmode='group',
    title='Зависимость тяжести ДТП от физического состояния водителя (количество)',
    histfunc='count',
    category_orders={
        'collision_damage': ['fatal', 'severe damage', 'middle damage', 'small damage', 'scratch']
    }
)
fig2 = px.histogram(
    df, x='party_drug_physical', color='collision_damage',
    barmode='group',
    title='Зависимость тяжести ДТП от физического состояния водителя (проценты)',
    histfunc='count',
    histnorm='percent',
    category_orders={
        'collision_damage': ['fatal', 'severe damage', 'middle damage', 'small damage', 'scratch']
    }
)
fig.show()
fig2.show()

Количество аварий с фатальными повреждениями ТС существенно выше в случае если водитель находится под воздействием лекарств (на них приходится больше 90% всех случаев), усталость и сонливость почти не влияют на риск ДТП с тяжёлыми последствиями. Количество ДТП с участниками под лекарственными веществами и в сонном состоянии в количественном соотношении равны. Количество ДТП связанных с ухудшением физического состояния ощутимо меньше. В целом соотношение типов ДТП совпадает за исключением процента тяжёлых транспортных проишествий.

## Создайте модель для оценки водительского риска

## Выявление признаков и выгрузка данных

Выгрузим данные по заданию заказчика:

тип виновника - car;

степень повреждений ТС - все кроме scratch;

дата столкновения - 2012 год.

In [ ]:
query = """
    SELECT
        c.weather_1,
        c.road_surface,
        c.road_condition_1,
        c.lighting,
        p.party_sobriety,
        p.party_drug_physical,
        p.cellphone_in_use,
        p.at_fault,
        p.case_id,
        v.vehicle_age,
        v.vehicle_transmission
    FROM
        collisions c
    JOIN
        parties p ON c.case_id = p.case_id
    JOIN
        vehicles v ON p.case_id = v.case_id AND p.party_number = v.party_number
    WHERE
        p.party_type = 'car'
        AND c.collision_damage != 'scratch'
        AND EXTRACT(YEAR FROM c.collision_date) = 2012
"""

df = pd.read_sql(query, engine)
df.head(5)

In [ ]:
df.info()

Учитывая набор данных и характер вождения, на вероятность аварии могут повлиять следующие факторы:

weather_1: погодные условия могут существенно повлиять на безопасность вождения;

road_surface: состояние дорожного покрытия может повлиять на сцепление и устойчивость;

road_condition_1: дорожные условия, такие как влажная, обледенелая или строящаяся дорога, могут влиять на риск вождения;

lighting: плохое освещение может ухудшить видимость и повысить риск несчастных случаев;

party_sobriety: уровень трезвости водителя является решающим фактором безопасности вождения;

party_drug_physical: влияние наркотиков или физическое состояние водителя могут ухудшить его способности к вождению;

cellphone_in_use: использование мобильного телефона во время вождения может отвлекать водителя;

vehicle_age: в старых автомобилях могут отсутствовать современные функции безопасности, что увеличивает риск;

vehicle_transmission: тип коробки передач может влиять на стиль вождения и время реакции.

Обработка пропущенных значений требует решений, специфичных для предметной области. Вот общий подход, который мы можем использовать, исходя из недостающего процента пропущенных значений:

Удаление столбцов. Для столбцов с высоким процентом пропущенных значений (например, party_drug_physical) может иметь смысл вообще удалить столбец, особенно если уровень пропущенных данных превышает 90%.

Заполнить с помощью наиболее частого значения: для категориальных переменных с более низким процентом пропущенных значений мы можем заполнить пропущенные значения с помощью mode (наиболее частого значения) столбца.

Заполнить медианой: для непрерывных переменных, таких как vehicle_age, мы можем заполнить недостающие значения медианой столбца.

Особые случаи: для таких столбцов, как cellphone_in_use, которые выглядят как двоичный индикатор (0 или 1), возможно, имеет смысл рассматривать пропущенные значения как «неизвестные» или заполнять специальным значением.

In [ ]:
df = df.drop(columns=['party_drug_physical'])

categorical_columns = ['weather_1', 'road_surface', 'road_condition_1', 'lighting', 'party_sobriety', 'vehicle_transmission']
for column in categorical_columns:
    mode_value = df[column].mode()[0]
    df[column] = df[column].fillna(mode_value)

df['vehicle_age'] = df['vehicle_age'].fillna(df['vehicle_age'].median())

# Для «cellphone_in_use» заполним пропущенные значения специальным значением (2), указывающим «неизвестно».
df['cellphone_in_use'] = df['cellphone_in_use'].fillna(2)

remaining_missing = df.isnull().sum()
remaining_missing

Проверим дубликаты.

In [ ]:
df.duplicated().sum()

Удалим дубликаты.

In [ ]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
df = df.drop(columns=['case_id'])

Переименуем столбцы к принятому виду.

In [ ]:
df.rename(columns={'weather_1':'weather', 'road_condition_1':'road_condition'}, inplace=True)

In [ ]:
my_report = sv.analyze(df)
my_report.show_notebook(  w=None,
                h=None,
                scale=None,
                layout='widescreen',
                filepath=None)

Выводы из графиков:

weather_1: Большинство несчастных случаев происходит в ясную погоду, что вполне ожидаемо, поскольку ясная погода является наиболее распространенным явлением;

road_surface: Судя по всему, на сухих дорогах происходит большинство аварий, что на первый взгляд может показаться нелогичным. Однако это может быть связано с тем, что большая часть вождения приходится на засушливые дни;

road_condition_1: Аналогично, большинство аварий происходит на дорогах без ям;

lighting: при дневном свете происходит наибольшее количество аварий, за ним следует темнота c уличным освещением;

party_sobriety: В большинстве аварий участвуют водители, не употреблявшие алкоголь. Однако это также может быть связано с тем, что большинство водителей на дороге трезвы;

cellphone_in_use: Небольшая часть ДТП связана с использованием мобильного телефона;

vehicle_transmission: Автомобили с автоматической коробкой передач участвуют в большем количестве аварий, но это может быть связано с преобладанием автомобилей с автоматической коробкой передач;

vehicle_age: В большинстве аварий участвуют автомобили возрастом от 0 до 10 лет, с пиком около 2-3 лет.

Наблюдения и корректировки:

Важными факторами могут быть погода, дорожное покрытие и освещение. Несмотря на то, что большинство аварий происходит в ясную погоду и на сухих дорогах, это связано с тем, что такие условия являются наиболее распространенными. Чтобы получить истинную картину, нам нужно будет нормализовать общее количество поездок в каждом состоянии;

Трезвость являются решающим фактором. Несмотря на то, что в большинстве аварий участвуют трезвые водители, важно понимать соотношение трезвых и нетрезвых водителей на дороге;

Использование мобильных телефонов, хотя и меньшинство, может быть важным фактором, учитывая потенциальное отвлечение;

Возраст автомобиля также может играть роль: новые автомобили могут иметь лучшие функции безопасности, но на них также чаще ездят.

In [ ]:
df.head(2)

In [ ]:
numeric = [
    'at_fault', 'vehicle_age', 'cellphone_in_use'
]

category = [
    'party_sobriety',
    'vehicle_transmission',
    'weather',
    'road_surface', 'lighting', 'road_condition'
]

df = df.astype({
    **{_:'int' for _ in numeric},
    **{_:'category' for _ in category}
})

In [ ]:
df.info()

In [ ]:
df_copy = df.copy()

Разделим данные на обучающие и тестовые для features(X) и target(y).

In [ ]:
X = df.drop(['at_fault'], axis=1)
y = df['at_fault']

X, X_test, y, y_test = train_test_split(
    X, y,
    test_size=.2,
    random_state=SEED,
    stratify=y
)

Проверим корректность разделения на выборки.

In [ ]:
print(f'Доля тренировочных признаков: {X.shape[0]/df.shape[0]:.2f}')
print(f'Доля тренировочных целей: {y.shape[0]/df.shape[0]:.2f}')
print(f'Доля тестовых признаков: {X_test.shape[0]/df.shape[0]:.2f}')
print(f'Доля тестовых целей: {y_test.shape[0]/df.shape[0]:.2f}')
print(f'Доля положительных целей на тренировочных данных: {y.astype("int").mean():.2f}')
print(f'Доля положительных целей на тестовых данных: {y_test.astype("int").mean():.2f}')

Выводы

Мы подготовили набор данных на основе требований заказчика. Обработали данные: убрали аномалии, почистили, заполнили где это возможно. Избавились от пропусков и дубликатов, поменяли типы данных на более рациональные, что позволило значительно уменьшить размер базы данных.

Можно переходить к работе с моделями машинного обучения.

## Найдём лучшую модель

Смоделируем не менее 3-х типов моделей с перебором гиперпараметров. 3 модели из классического машинного обучения; 1 модель из бустингов. Оформим вывод в виде сравнительной таблицы.

В данном исследовании стоит задача бинарной классификации, для неё подходят следующие метрики оценки качества моделей: precision, recall и f1 мера.

Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными.

Recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.

Так как у нас необходимо угадать будет ли являться наш клиент виновником проишествия, то мы выберем метрику f1 как основную, так как она связывает отношение между precision и recall.

### DummyClassifier

In [ ]:
%%time

model_dc = DummyClassifier(strategy='most_frequent')
model_dc.fit(X, y)
prediction_dc = model_dc.predict(X_test)

In [ ]:
cr_dc = classification_report(
    y_test, prediction_dc,
    output_dict=True, zero_division=0
)

results = {} # словарь со сводными результатами для всех моделей
results['Dummy Classifier (most frequent)'] = cr_dc['weighted avg']

pd.DataFrame(cr_dc).round(decimals=3).transpose()

Результат неудовлетворительный. Посмотрим на модель логистической регрессии.

### LogisticRegression

Создадим трансформер для различных колонок, в случае с "линейными" моделями к категориальным применим OHE, к числовым - StandardScaler.

In [ ]:
linear_transformer = make_column_transformer(
    (
        OneHotEncoder(
            dtype='uint8',
            handle_unknown='ignore', drop='first'
        ), make_column_selector(dtype_include=['category', 'object'])
    ),
    (
        StandardScaler(
        ), make_column_selector(dtype_include='number')
    ),
    remainder='passthrough'
)

Соберём pipeline для линейной регрессии.

In [ ]:
pipe_lr = make_pipeline(
    linear_transformer,
    LogisticRegression(
        class_weight='balanced',
        random_state=SEED,
        n_jobs=-1
    )
)

Зададим сетку параметров и создадим estimator.

In [ ]:
scoring = {'PRECISION': 'precision', 'F1': 'f1', 'RECALL': 'recall'}

param_grid = {
    'logisticregression__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__C':[.001, .01],
    'logisticregression__max_iter':[500, 1000]
}

gs_lr = GridSearchCV(
    pipe_lr,
    param_grid,
    scoring=scoring,
    refit = 'F1',
    return_train_score=True,
    n_jobs=-1,
    verbose=1
)

Проведём обучение с кросс-валидацией выбранного оценщика.

In [ ]:
%time gs_lr.fit(X, y)
None

In [ ]:
best_params = gs_lr.best_params_
best_estimator = gs_lr.best_estimator_
best_score = gs_lr.best_score_
print(best_params)
print(best_estimator) 
print(best_score)  

In [ ]:
scores = pd.DataFrame(gs_lr.cv_results_)
scores.head(2)

In [ ]:
print(gs_lr.cv_results_['split0_test_PRECISION'][gs_lr.best_index_], gs_lr.cv_results_['mean_test_RECALL'][gs_lr.best_index_], gs_lr.cv_results_['split0_train_RECALL'][gs_lr.best_index_])

Посмотрим на результаты лучшей модели и залогируем результат.

### RandomForestClassifier

Создадим трансформер для различных колонок, в случае с "деревянными моделями" к категориальным применим OE, числовые оставим как есть.

In [ ]:
ensemble_transformer = make_column_transformer(
    (
        OrdinalEncoder(
            dtype='int16',
            handle_unknown='use_encoded_value',
            unknown_value=-1
        ), make_column_selector(dtype_include=['category', 'object'])
    ),
    remainder='passthrough'
)

Создадим pipeline для классификатора RandomForestClassifier().

In [ ]:
pipe_rf = make_pipeline(
    ensemble_transformer,
    RandomForestClassifier(
        random_state=SEED,
        n_jobs=-1,
        class_weight='balanced',
        verbose=0
    )
)

Зададим сетку параметров и создадим estimator.

In [ ]:
param_grid = {
    'randomforestclassifier__n_estimators':np.arange(100, 151, 50),
    'randomforestclassifier__max_depth':np.arange(1, 10, 2),
}

gs_rf = GridSearchCV(
    pipe_rf,
    param_grid,
    scoring=scoring,
    refit = 'F1',
    return_train_score=True,
    n_jobs=-1,
    verbose=1
)

Проведём обучение с кросс-валидацией выбранного оценщика.

In [ ]:
%time gs_rf.fit(X, y)
None

Посмотрим на результаты лучшей модели и залогируем результат.

In [ ]:
best_params = gs_rf.best_params_
best_estimator = gs_rf.best_estimator_
best_score = gs_rf.best_score_
print(best_params)
print(best_estimator) 
print(best_score)  

In [ ]:
scores = pd.DataFrame(gs_rf.cv_results_)
scores.head(2)

In [ ]:
print(gs_rf.cv_results_['split0_test_PRECISION'][gs_rf.best_index_], gs_rf.cv_results_['mean_test_RECALL'][gs_rf.best_index_], gs_rf.cv_results_['split0_train_RECALL'][gs_rf.best_index_])

Результаты сходные с логистической регрессией.

### LightGBM

Для работы с библиотекой необходимо перевести все категориальные признаки в целые числа. Воспользуемся определённым ранее трансформером для ансамблей.

Создадим pipeline для классификатора LightGBM.

In [ ]:
pipe_gbm = make_pipeline(
    ensemble_transformer,
    lgb.LGBMClassifier(
        objective='binary',
        metric='f1',
        n_jobs=-1,
        verbosity=-1,
        random_state=SEED
    )
)

Зададим сетку параметров и создадим estimator.

In [ ]:
param_grid = {
    'lgbmclassifier__max_depth':[-1, 10, 20],
    'lgbmclassifier__num_leaves':[10, 31],
    'lgbmclassifier__learning_rate':[.1, .5]
}

gs_gbm = GridSearchCV(
    pipe_gbm,
    param_grid,
    scoring=scoring,
    refit = 'F1',
    return_train_score=True,
    n_jobs=-1,
    verbose=1
)

Проведём обучение с кросс-валидацией выбранного оценщика.

In [ ]:
%time gs_gbm.fit(X, y)
None

In [ ]:
best_params = gs_gbm.best_params_
best_estimator = gs_gbm.best_estimator_
best_score = gs_gbm.best_score_
print(best_params)
print(best_estimator) 
print(best_score)  

In [ ]:
scores = pd.DataFrame(gs_gbm.cv_results_)
scores.head(2)

In [ ]:
print(gs_gbm.cv_results_['split0_test_PRECISION'][gs_gbm.best_index_], gs_gbm.cv_results_['mean_test_RECALL'][gs_gbm.best_index_], gs_rf.cv_results_['split0_train_RECALL'][gs_gbm.best_index_])

По показателям метрик видно, что лучше всех с нашей задачей справился бустинг LightGBM. Выберем модель бустинга, как наилучшую.

### Проверим лучшую модель в работе

Проведём графический анализ «Матрица ошибок». Выведем полноту и точность на график.

In [ ]:
prediction_gbm = gs_gbm.predict(X_test)

cr_gbm = classification_report(
    y_test, prediction_gbm,
    output_dict=True, zero_division=0
)

In [ ]:
cm = confusion_matrix(y_test, prediction_gbm, labels=gs_gbm.classes_)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
cmd = ConfusionMatrixDisplay(cm, display_labels=gs_gbm.classes_)
cmd.plot(ax=ax1)
ax1.set_title('Матрица ошибок')
prec, recall, _ = precision_recall_curve(y_test, prediction_gbm)
PrecisionRecallDisplay(prec, recall).plot(ax=ax2)
ax2.set_title('График полнота-точность')
plt.show()

Матрица ошибок позволяет оценить, как модель классифицирует положительные и отрицательные случаи. Зная эти значения, можно вычислить другие метрики, такие как полнота, точность, специфичность и F1-мера, для оценки производительности модели.

Значения в матрице ошибок:

Верхняя левая ячейка | True Negative (TN=4132) - количество правильно классифицированных отрицательных случаев. Оно указывает на количество случаев, которые модель правильно предсказала как отрицательные, когда они действительно являются отрицательными.

Верхняя правая ячейка | False Positive (FP=1412) - количество неправильно классифицированных положительных случаев. Это количество случаев, которые модель неправильно предсказала как положительные, когда они являются отрицательными.

Нижняя левая ячейка | False Negative (FN=2875) - количество неправильно классифицированных отрицательных случаев. Оно указывает на количество случаев, которые модель неправильно предсказала как отрицательные, когда они являются положительными.

Нижняя правая ячейка | True Positive (TP=2800) - количество правильно классифицированных положительных случаев. Оно указывает на количество случаев, которые модель правильно предсказала как положительные, и они действительно являются положительными.

Модель эффективно идентифицирует истинно отрицательные случаи (36%), однако допускает значительное количество ложноотрицательных (25%) и ложноположительных (13%) ошибок. Точность истинноположительных предсказаний составляет 26%, что указывает на потенциальные слабые стороны в распознавании положительных классов. Такая ситуация может потребовать применения более сложных моделей для улучшения точности предсказаний.

## Проведите анализ важности факторов ДТП

Для анализа факторов влияющих на вероятность стать виновником ДТП воспользуемся библиотекой SHAP.

In [ ]:
X_encoded = gs_gbm.best_estimator_['columntransformer'].fit_transform(X)
model = gs_gbm.best_estimator_['lgbmclassifier'].fit(X_encoded, y)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_encoded)

shap.summary_plot(shap_values, X, show=False)
plt.title('Важность признаков (SHAP)')
plt.ylabel("Признаки")
plt.show()

In [ ]:
X_encoded = gs_gbm.best_estimator_['columntransformer'].fit_transform(X)
model = gs_gbm.best_estimator_['lgbmclassifier'].fit(X_encoded, y)
# инициализация Explainer
explainer = shap.Explainer(model)

# получение значений SHAP для тестовых данных
shap_values = explainer.shap_values(X_test)

# визуализация SHAP значений
shap.summary_plot(shap_values, X_test, plot_type='bar', show=False, color='teal')
plt.title('Важность признаков (SHAP)')
plt.ylabel("Признаки")
plt.show()

Как видно из графика, самым главным признаком, влияющий на фак аварии является party_sobriety - трезвость участника, при предобработки данных мы разделили записи в зависимости от степени опьянения. Посмотрим зависимость вероятности стать виновником ДТП от уровня опьянения:

Посмотрим уникальные значения в столбце и распределим их по степени опьянения.

In [ ]:
df['party_sobriety'].unique()

In [ ]:
def replace_sobriety(x):
    if x in ['had not been drinking', 'impairment unknown', 'not applicable']: return 0
    elif x == 'had been drinking, not under influence': return 1
    elif x == 'had been drinking, impairment unknown' : return 2
    else: return 3

df['party_sobriety'] = df['party_sobriety'].apply(replace_sobriety)

In [ ]:
# сводная таблица для среднего значения 'at_fault' по уровню опьянения 'party_sobriety'
mean_at_fault = df.groupby('party_sobriety')['at_fault'].mean()

# построение графика
sns.set_style("whitegrid")
plt.figure(figsize=(8, 4))
sns.lineplot(x=mean_at_fault.index, y=mean_at_fault.values, marker='o', color='b')
plt.xlabel('Уровень опьянения (party_sobriety)')
plt.ylabel('Среднее значение at_fault')
plt.title('Зависимость вероятности стать виновником ДТП от уровня опьянения')
plt.xticks(mean_at_fault.index)
plt.show()

Из графика видно, что с увеличением уровня опьянения (от 0 до 3) вероятность стать виновником ДТП увеличивается. Чем выше уровень опьянения, тем больше вероятность попадания в аварию, где водитель считается виновным.

Чтобы учесть этот фактор во время посадки водителя, можно использовать различные технические средства и системы, такие как:

Алкотестеры: Установка алкотестеров в автомобиле позволит водителю проверить свой уровень опьянения перед поездкой и принять решение о том, стоит ли ему садиться за руль.

Автоматические системы блокировки двигателя: Эти системы могут быть настроены на блокировку двигателя, если уровень опьянения водителя превышает определенный уровень.

Системы предупреждения о состоянии водителя: Использование камер и сенсоров для мониторинга поведения водителя и выявления признаков усталости или опьянения.

Эти технологии и системы могут помочь снизить вероятность аварий, связанных с опьянением водителей, и повысить безопасность дорожного движения. Однако следует помнить, что самое эффективное средство предотвращения аварий - это не садиться за руль в состоянии опьянения.

## Выводы

В данной работе мы попытались создать систему для каршеринговой компании, которая могла бы оценить риск ДТП по совокупности факторов.

Мы исследовали предоставленную базу, провели статистический анализ некоторых факторов.

Подготовили данные для анализа за 2012 год. Опробовали ряд моделей классического машинного обучения и бустингов.

Испытанные модели показали, что внедрение данных систем позволит в некоторой степени улучшить вероятность опознать потенциального виновника ДТП по ряду признаков.

Мы увидели что самыми влияющими факторами являются :

трезвость и состояние водителя,

время суток,
состояние дороги и погодные условия.


Общий вклад трезвости довольно велик и его стоит учитывать. В машины стоит установить контрольное устройство. Возможно, добавить в камеру систему определения физического состояния водителя по времени реакции на раздражители, скорость моргания и сфокусированность зрения на дороге с включением звуковой сигнализации если водитель начинает спать или долго не смотрит на дорогу.

Нам удалось обучить модель на тестовых данных и получить метрику Recall равную 62%, precision - 59%,  Это недостаточно хороший показатель, однако на предсказаниях мы имеем:

неправильно угадываем парядка 24% данных, которые относятся к первому классу
чаще всего мы будем угадывать класс 1 - в районе 29% данных. Это не может не радовать.
класс 0 мы будем угадывать в 24% от случая.
а наименьший процент из всех представленных - неудачные предсказания класса 0 (19%)
Если отвечать на первоначальный вопрос: "Возможно ли прогнозировать?", я бы сказал нет, точнее, не на этих данных. Будем честны, большинство факторов из выборки будут известны уже после проишествия. Нам же нужны совсем иные данные, которые мы будем знать только, когда человек сел в машину и выбрал маршрут. К примеру, я бы добавил в таблицу характеристик клиента следующие факторы:

возраст водителя, который будет прямо влиять на время реакции,
стаж, от этого будет зависить как он поведёт себя в той или иной ситуации,
среднюю сокорость вождения,
количество нарушений за всё время за рулём,
возможно, можно провести исследования и добавить сюда место работы.